In [25]:
#tutorial https://medium.com/omics-diary/how-to-use-the-lazy-predict-library-to-select-the-best-machine-learning-model-65378bf4568e
!pip install lazypredict
!pip install scikit-learn==1.3.2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [26]:
df_l_1_path = "./datasets_no_data_smells/kdd-census.csv"  #69850 rows, categorie sono 0 e 1, 'income_level' 7.5% di label 1
df_l_2_path = "./datasets_no_data_smells/diabetic_data.csv" #101766 rows categorie sono NO, >30 e <30
df_m_1_path = "./datasets_no_data_smells/Firefighter_Promotion_Exam_Scores.csv" #118 rows, "promotion" e "no promotion"
df_l_3_path = "./datasets_no_data_smells/nursery.csv" #12960 rows, recommend e not_recommend
df_m_2_path = "./datasets_no_data_smells/tae.csv" #151 rows, classi 3 2 e 1
df_l_1 = pd.read_csv(df_l_1_path, index_col=False)
df_l_2 = pd.read_csv(df_l_2_path, index_col=False)
df_m_1 = pd.read_csv(df_m_1_path, index_col=False)
df_l_3 = pd.read_csv(df_l_3_path, index_col=False)
df_m_2 = pd.read_csv(df_m_2_path, index_col=False)

Estraiamo dei dataset più piccoli dei due dataset kdd-census e diabetic_data  per poterli esaminare con il tool DSD

In [27]:
!mkdir smaller_datasets
df_l_1.iloc[:34926].to_csv("smaller_datasets/kdd-census_1.csv", index = False)
df_l_1.iloc[34926:69852].to_csv("smaller_datasets/kdd-census_2.csv", index = False)
df_l_2.iloc[:33923].to_csv("smaller_datasets/diabetic_data_1.csv", index = False)
df_l_2.iloc[33923:67846].to_csv("smaller_datasets/diabetic_data_2.csv", index = False)
df_l_2.iloc[67846:101769].to_csv("smaller_datasets/diabetic_data_3.csv", index = False)

mkdir: cannot create directory ‘smaller_datasets’: File exists


Data smells (tra quelli in analisi) sui dataset kdd-census e diabetic_data (calcolati sui sotto-dataset righe)
kdd-census: Missing Value Smells in migration_msa, migration_reg, migration_sunbelt, migration_within_reg


diabetic_datasets: Missing Value Smells in A1Cresult, max_glu_serum
Casing Smells in medical_specialty, race

In [28]:
df_l_1 = df_l_1.drop(columns = ['migration_msa', 'migration_reg', 'migration_sunbelt', 'migration_within_reg'])
df_l_2 = df_l_2.drop(columns = ['A1Cresult', 'max_glu_serum', 'medical_specialty', 'race'])

In [29]:
columns = []
for i in df_l_1.columns:
  if df_l_1[i].nunique() == len(df_l_1):
    columns.append(i)
columns

[]

In [30]:
columns = []
for i in df_l_2.columns:
  if df_l_2[i].nunique() == len(df_l_2):
    columns.append(i)
columns

['encounter_id']

In [31]:
columns = []
for i in df_l_3.columns:
  if df_l_3[i].nunique() == len(df_l_3):
    columns.append(i)
columns

['Unnamed: 0']

In [32]:
columns = []
for i in df_m_1.columns:
  if df_m_1[i].nunique() == len(df_m_1):
    columns.append(i)
columns

[]

In [33]:
columns = []
for i in df_m_2.columns:
  if df_m_2[i].nunique() == len(df_m_2):
    columns.append(i)
columns

[]

Rimuoviamo le feature individuate che rappresentano id unici, da rimuovere in fase di pre-processing

In [34]:
df_l_2 =  df_l_2.drop(columns=['encounter_id'])
df_l_3 =  df_l_3.drop(columns=['Unnamed: 0'])

Salviamo i nuovi dataset, anche quelli non modificati. Salviamo anche degli split di dimensioni ridotte per i primi due dataset, da poter poi fornire in input a DSD Tool

In [35]:
!mkdir cleaned_datasets

df_l_1.to_csv("cleaned_datasets/df_l_1.csv", index = False)
df_l_2.to_csv("cleaned_datasets/df_l_2.csv", index = False)
df_l_3.to_csv("cleaned_datasets/df_l_3.csv", index = False)
df_m_1.to_csv("cleaned_datasets/df_m_1.csv", index = False)
df_m_2.to_csv("cleaned_datasets/df_m_2.csv", index = False)
df_l_1.iloc[:34926].to_csv("cleaned_datasets/df_l_1_1.csv", index = False)
df_l_1.iloc[34926:69852].to_csv("cleaned_datasets/df_l_1_2.csv", index = False)
df_l_2.iloc[:33923].to_csv("cleaned_datasets/df_l_2_1.csv", index = False)
df_l_2.iloc[33923:67846].to_csv("cleaned_datasets/df_l_2_2.csv", index = False)
df_l_2.iloc[67846:101769].to_csv("cleaned_datasets/df_l_2_3.csv", index = False)

mkdir: cannot create directory ‘cleaned_datasets’: File exists


I dataset vengono pre-processati trasformando le feature categoriche in numeriche

In [36]:
categorical_l_1 = df_l_1.select_dtypes(include="object")
for i in categorical_l_1:
  df_l_1[i] = pd.factorize(df_l_1[i])[0]

categorical_l_2 = df_l_2.select_dtypes(include="object")
for i in categorical_l_2:
  df_l_2[i] = pd.factorize(df_l_2[i])[0]

categorical_l_3 = df_l_3.select_dtypes(include="object")
for i in categorical_l_3:
  df_l_3[i] = pd.factorize(df_l_3[i])[0]

categorical_m_1 = df_m_1.select_dtypes(include="object")
for i in categorical_m_1:
  df_m_1[i] = pd.factorize(df_m_1[i])[0]

categorical_m_2 = df_m_2.select_dtypes(include="object")
for i in categorical_m_2:
  df_m_2[i] = pd.factorize(df_m_2[i])[0]

Salviamo i dataset pre-processati.
Per "pre-processed" si intendono i dati a valle delle operazioni di fattorizzazione dei valori categorici in numerici.

In [37]:
!mkdir pre-processed_df
df_l_1.iloc[:34926].to_csv("pre-processed_df/kdd-census_1.csv", index = False)
df_l_1.iloc[34926:69852].to_csv("pre-processed_df/kdd-census_2.csv", index = False)
df_l_2.iloc[:33923].to_csv("pre-processed_df/diabetic_data_1.csv", index = False)
df_l_2.iloc[33923:67846].to_csv("pre-processed_df/diabetic_data_2.csv", index = False)
df_l_2.iloc[67846:101769].to_csv("pre-processed_df/diabetic_data_3.csv", index = False)

mkdir: cannot create directory ‘pre-processed_df’: File exists


Feature importance calcolata con ReliefFAttributeEval di Weka, che valuta quanto una feature è utile a distinguere una istanza dalle altri istanze simili. Sono calcolate sui dataset pre-processed.

Riferimenti:
https://weka.sourceforge.io/doc.dev/weka/attributeSelection/ReliefFAttributeEval.html

https://www.sciencedirect.com/science/article/pii/S1532046418301400




df_l_1 (kdd-census) Senza feature con data smells - 10 nearest neighbours
1. age - numerico int
2. industry_code - numerico int
3. major_industry_code - categ.
4. education - categ.
5. occupation_code - numerico int
6. class_of_worker - categ.
7. num_person_Worked_employer - numerico int
8. capital_gains - numerico int
9. capital_losses - numerico int
10. member_of_labor_union - categ.
11. business_or_self_employed - numerico int
12. major_occupation_code - categ.
13. tax_filer_status - categ.
14. dividend_from_Stocks - numerico int
15. marital_status - categ.
16. sex - categ.
17. full_parttime_employment_stat - categ.
18. citizenship - categ.
19. country_father - categ.
20. country_mother - categ.
21. wage_per_hour - numerico int
22. fill_questionnaire_veteran_admin - categ.
23. veterans_benefits - numerico int
24. d_household_summary - categ.
25. country_self - categ.
26. live_1_year_ago - categ.
27. year - numerico int
28. region_of_previous_residence - categ.
29. race - categ.
30. state_of_previous_residence - categ.
31. d_household_family_stat - categ.
32. weeks_worked_in_year - numerico int
33. reason_for_unemployment - categ.
34. hispanic_origin - categ.
35. enrolled_in_edu_inst_lastwk - categ.
36. family_members_under_18 - categ.

In [38]:
df_l_1.describe()

,age,class_of_worker,industry_code,occupation_code,education,wage_per_hour,enrolled_in_edu_inst_lastwk,marital_status,major_industry_code,major_occupation_code,...,country_father,country_mother,country_self,citizenship,business_or_self_employed,fill_questionnaire_veteran_admin,veterans_benefits,weeks_worked_in_year,year,income_level
count,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,...,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00,69850.00
mean,34.79,2.09,15.60,11.39,8.55,54.69,1.72,2.13,9.84,6.11,...,5.30,5.44,2.70,0.73,0.18,0.36,1.53,23.61,94.50,0.07
std,22.21,2.66,18.10,14.46,7.55,272.94,1.05,2.63,11.29,6.24,...,11.85,12.13,9.43,1.37,0.56,0.55,0.84,24.45,0.50,0.26
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.00,-1.00,-1.00,0.00,0.00,0.00,0.00,0.00,94.00,0.00
25%,16.00,1.00,0.00,0.00,3.00,0.00,1.00,0.00,2.00,2.00,...,1.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00,94.00,0.00
50%,33.00,1.00,1.00,2.00,4.00,0.00,1.00,1.00,3.00,3.00,...,1.00,1.00,0.00,0.00,0.00,0.00,2.00,12.00,94.00,0.00
75%,50.00,3.00,33.00,26.00,14.00,0.00,3.00,3.00,15.00,9.00,...,2.00,2.00,1.00,1.00,0.00,1.00,2.00,52.00,95.00,0.00
max,90.00,17.00,51.00,46.00,33.00,9900.00,5.00,13.00,47.00,29.00,...,84.00,84.00,84.00,9.00,2.00,5.00,2.00,52.00,95.00,1.00


df_l_2 (diabetic_data) SENZA ENCOUNTER_ID e Feature con data smells - 10 nearest neighbours

 1. number_inpatient - numerico int
 2. discharge_disposition_id - numerico int
 3. time_in_hospital - numerico int
 4. patient_nbr - numerico int
 5. number_outpatient - numerico int
 6. admission_source_id - numerico int
 7. num_medications - numerico int
 8. number_emergency - numerico int
 9. weight - categ. ma con solo numeri e parentesi
 10. admission_type_id - numerico int
 11. insulin - categ.
 12. pioglitazone - categ.
 13. num_lab_procedures - numerico int
 14. diag_3 - categ.
 15. repaglinide - categ.
 16. glipizide - categ.
 17. glyburide-metformin - categ.
 18. nateglinide - categ.
 19. miglitol - categ.
 20. diabetesMed - categ.
 21. troglitazone - categ.
 22. glimepiride-pioglitazone - categ.
 23. acetohexamide - categ.
 24. tolazamide - categ.
 25. metformin-rosiglitazone - categ.
 26. citoglipton - categ.
 27. examide - categ.
 28. glipizide-metformin - categ.
 29. tolbutamide - categ.
 30. metformin-pioglitazone - categ.
 31. change - categ.
 32. chlorpropamide - categ.
 33. acarbose - categ.
 34. metformin - categ.
 35. diag_1 - categ.
 36. rosiglitazone - categ.
 37. glimepiride - categ.
 38. glyburide - categ.
 39. num_procedures - numerico int
 40. diag_2 - categ.
 41. gender - categ.
 42. age - categ.
 43. payer_code - categ.
 44. number_diagnoses - numerico int


In [39]:
df_l_2.describe()

,patient_nbr,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
count,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,...,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00,101766.00
mean,54330400.69,0.46,6.10,0.07,2.02,3.72,5.75,4.40,1.89,43.10,...,0.00,1.08,0.01,0.00,0.00,0.00,0.00,0.46,0.77,0.57
std,38696359.35,0.50,1.59,0.50,1.45,5.28,4.06,2.99,2.87,19.67,...,0.00,1.11,0.09,0.01,0.00,0.00,0.00,0.50,0.42,0.68
min,135.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,23413221.00,0.00,5.00,0.00,1.00,1.00,1.00,2.00,0.00,31.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
50%,45505143.00,0.00,6.00,0.00,1.00,1.00,7.00,4.00,1.00,44.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
75%,87545949.75,1.00,7.00,0.00,3.00,4.00,7.00,6.00,3.00,57.00,...,0.00,2.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
max,189502619.00,2.00,9.00,9.00,8.00,28.00,25.00,14.00,17.00,132.00,...,0.00,3.00,3.00,1.00,1.00,1.00,1.00,1.00,1.00,2.00


df_l_3 (Nursery) SENZA FEATURE UNNAMED: 0
1. health - categ.
2. finance - categ.
3. parents - categ.
4. form - categ.
5. children - categ.
6. housing - categ.
7. social - categ.
8. has_nurs - categ.

In [40]:
df_l_3.describe()

,parents,has_nurs,form,children,housing,finance,social,health,class
count,12960.00,12960.00,12960.00,12960.00,12960.00,12960.00,12960.00,12960.00,12960.00
mean,1.00,2.00,1.50,1.50,1.00,0.50,1.00,1.00,0.33
std,0.82,1.41,1.12,1.12,0.82,0.50,0.82,0.82,0.47
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,1.00,0.75,0.75,0.00,0.00,0.00,0.00,0.00
50%,1.00,2.00,1.50,1.50,1.00,0.50,1.00,1.00,0.00
75%,2.00,3.00,2.25,2.25,2.00,1.00,2.00,2.00,1.00
max,2.00,4.00,3.00,3.00,2.00,1.00,2.00,2.00,1.00


df_m_1 (Firefighter)
 1. combine - numerico float
 2. written - numerico int
 3. oral - numerico float
 4. position - categ.
 5. race - categ.

In [41]:
df_m_1.describe()

,position,oral,written,race,combine,promotion
count,118.00,118.00,118.00,118.00,118.00,118.00
mean,0.65,65.52,71.65,0.65,69.20,0.53
std,0.48,12.42,10.62,0.83,9.39,0.50
min,0.00,40.83,46.00,0.00,45.93,0.00
25%,0.00,55.39,65.00,0.00,60.94,0.00
50%,1.00,66.04,71.50,0.00,69.50,1.00
75%,1.00,73.81,78.50,1.00,75.83,1.00
max,1.00,92.08,95.00,2.00,92.81,1.00


df_m_2 (tae)
1. Course - numerico int
2. Class size - numerico int
3. Course instructor - numerico int
4. Summer or regular semester - numerico int
5. English speaker - numerico int

In [42]:
df_m_2.describe()

,English_speaker,Course_instructor,Course,Summer_or_regular_semester,Class_size,Class_attribute
count,151.00,151.00,151.00,151.00,151.00,151.00
mean,1.81,13.64,8.11,1.85,27.87,2.02
std,0.40,6.83,7.02,0.36,12.89,0.82
min,1.00,1.00,1.00,1.00,3.00,1.00
25%,2.00,8.00,3.00,2.00,19.00,1.00
50%,2.00,13.00,4.00,2.00,27.00,2.00
75%,2.00,20.00,15.00,2.00,37.00,3.00
max,2.00,25.00,26.00,2.00,66.00,3.00


In [43]:
df_m_2

,English_speaker,Course_instructor,Course,Summer_or_regular_semester,Class_size,Class_attribute
0,1,23,3,1,19,3
1,2,15,3,1,17,3
2,1,23,3,2,49,3
3,1,5,2,2,33,3
4,2,7,11,2,55,3
...,...,...,...,...,...,...
146,2,3,2,2,26,1
147,2,10,3,2,12,1
148,1,18,7,2,48,1
149,2,22,1,2,51,1


Creiamo dei campioni da dare a ChatGPT per generare nuove istanze con data smells, in modo da mantenere le proporzioni delle label nel dataset

In [44]:
x_l_1, _, y_l_1, _= train_test_split(df_l_1.drop('income_level', axis=1), df_l_1['income_level'], train_size=25, stratify=df_l_1['income_level'], random_state=42)
x_l_1["income_level"]=y_l_1
!mkdir samples
x_l_1.to_csv("samples/x_l_1.csv", index=False)

x_l_2, _, y_l_2, _= train_test_split(df_l_2.drop('readmitted', axis=1), df_l_2['readmitted'], train_size=25, stratify=df_l_2['readmitted'], random_state=42)
x_l_2["readmitted"]=y_l_2
x_l_2.to_csv("samples/x_l_2.csv", index=False)

x_l_3, _, y_l_3, _= train_test_split(df_l_3.drop('class', axis=1), df_l_3['class'], train_size=25, stratify=df_l_3['class'], random_state=42)
x_l_3["class"]=y_l_3
x_l_3.to_csv("samples/x_l_3.csv", index=False)

x_m_1, _, y_m_1, _= train_test_split(df_m_1.drop('promotion', axis=1), df_m_1['promotion'], train_size=25, stratify=df_m_1['promotion'], random_state=42)
x_m_1["promotion"]=y_m_1
x_m_1.to_csv("samples/x_m_1.csv", index=False)

x_m_2, _, y_m_2, _= train_test_split(df_m_2.drop('Class_attribute', axis=1), df_m_2['Class_attribute'], train_size=25, stratify=df_m_2['Class_attribute'], random_state=42)
x_m_2["Class attribute"]=y_m_2
x_m_2.to_csv("samples/x_m_2.csv", index=False)